In [ ]:
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!wget https://www.manythings.org/anki/rus-eng.zip
!unzip rus-eng.zip
!tail rus.txt

--2023-06-23 06:42:43--  https://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15460248 (15M) [application/zip]
Saving to: ‘rus-eng.zip’

rus-eng.zip         100%[===================>]  14.74M  50.6MB/s    in 0.3s    

2023-06-23 06:42:44 (50.6 MB/s) - ‘rus-eng.zip’ saved [15460248/15460248]

Archive:  rus-eng.zip
  inflating: rus.txt                 
  inflating: _about.txt              
We need to uphold laws against discrimination — in hiring, and in housing, and in education, and in the criminal justice system. That is what our Constitution and our highest ideals require.	Нам нужно отстаивать законы против дискриминации при найме на работу, в жилищной сфере, в сфере образования и правоохранительной системе. Этого требуют наша Конституция и высшие идеалы.	CC-BY 2.0 (France) Attribution

In [ ]:
SOS_token = 0
EOS_token = 1


class Lang:
  def __init__(self, name):
    self.name = name
    self.word2index = {}
    self.word2count = {}
    self.index2word = {0: 'SOS', 1: 'EOS'}
    self.n_words = 2

  def add_sentence(self, sentence):
    for word in sentence.split(' '):
      self.add_word(word)

  def add_word(self, word):
    if word not in self.word2index:
      self.word2index[word] = self.n_words
      self.word2count[word] = 1
      self.index2word[self.n_words] = word
      self.n_words += 1
    else:
      self.word2count[word] += 1

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Zа-яА-Я.!?]+", r" ", s)
    return s

In [ ]:
def read_lang(lang1='rus', lang2='eng', reverse=False):
  with open('rus.txt', encoding='utf-8') as f:
    data = f.readlines()

  pairs = [[normalizeString(j) for j in i.strip().split('\t')[:2]] for i in data]

  if reverse:
    pairs = [list(reversed(p)) for p in pairs]
    input_lang = Lang(lang2)
    output_lang = Lang(lang1)
  else:
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)
  return input_lang, output_lang, pairs

In [ ]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[0].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = read_lang(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.add_sentence(pair[0])
        output_lang.add_sentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'rus')
print(random.choice(pairs))

Read 467119 sentence pairs
Trimmed to 27228 sentence pairs
Counting words...
Counted words:
eng 4229
rus 9939
['i m not sure tom is ready .', 'я не уверен что том готов .']


## Encoder

In [ ]:
class EncoderRNN(nn.Module):
  def __init__(self, input_size, hidden_size, rnn_type, num_layers):
    super(EncoderRNN, self).__init__()
    self.num_layers = num_layers
    self.hidden_size = hidden_size

    self.embedding = nn.Embedding(input_size, hidden_size)
    self.rnn = rnn_type(hidden_size, hidden_size, num_layers, batch_first=True)

  def forward(self, input, hidden):

    embedded = self.embedding(input).view(1, 1, -1)
    output, hidden = self.rnn(embedded, hidden)
    return output, hidden

  def initHidden(self):
    return torch.zeros(self.num_layers, 1, self.hidden_size, device=device)

## Decoder

In [ ]:
class DecoderRNN(nn.Module):
  def __init__(self, hidden_size, output_size, rnn_type, num_layers, dropout=0.1, max_length=MAX_LENGTH):
    super(DecoderRNN, self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.num_layers = num_layers
    self.dropout_p = dropout
    self.max_length = max_length

    self.embedding = nn.Embedding(self.output_size, self.hidden_size)

    self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
    self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
    self.dropout = nn.Dropout(self.dropout_p)

    self.rnn = rnn_type(self.hidden_size, self.hidden_size, self.num_layers, batch_first=True)
    self.fc = nn.Linear(self.hidden_size, self.output_size)
#     self.softmax = nn.LogSoftmax(dim=1)

  def forward(self, input, hidden, encoder_outputs):
    embedded = self.embedding(input).view(1, 1, -1)
    embedded = self.dropout(embedded)

    attn_weights = F.softmax(
        self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)

    attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                            encoder_outputs.unsqueeze(0))

    output = torch.cat((embedded[0], attn_applied[0]), 1)
    output = self.attn_combine(output).unsqueeze(0)

    output = F.relu(output)
    output, hidden = self.rnn(output, hidden)
    output = F.log_softmax(self.fc(output[0]), dim=1)
    return output, hidden, attn_weights

  def initHidden(self):
    return torch.zeros(self.num_layers, 1, self.hidden_size, device=device)

## Decoder scalar

In [ ]:
class DecoderRNN_scalar(nn.Module):
  def __init__(self, hidden_size, output_size, rnn_type, num_layers, dropout=0.1, max_length=MAX_LENGTH):
    super(DecoderRNN_scalar, self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.num_layers = num_layers
    self.dropout_p = dropout
    self.max_length = max_length

    self.embedding = nn.Embedding(self.output_size, self.hidden_size)

    self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
    self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
    self.dropout = nn.Dropout(self.dropout_p)

    self.rnn = rnn_type(self.hidden_size, self.hidden_size, self.num_layers, batch_first=True)
    self.fc = nn.Linear(self.hidden_size, self.output_size)
#     self.softmax = nn.LogSoftmax(dim=1)

  def forward(self, input, hidden, encoder_outputs):
    embedded = self.embedding(input).view(1, 1, -1)
    embedded = self.dropout(embedded)

    attn_weights = F.softmax( (embedded[0] @ encoder_outputs.T) / self.max_length**0.5, dim=1)

    attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                            encoder_outputs.unsqueeze(0))

    output = torch.cat((embedded[0], attn_applied[0]), 1)
    output = self.attn_combine(output).unsqueeze(0)

    output = F.relu(output)
    output, hidden = self.rnn(output, hidden)
    output = F.log_softmax(self.fc(output[0]), dim=1)
    return output, hidden, attn_weights

  def initHidden(self):
    return torch.zeros(self.num_layers, 1, self.hidden_size, device=device)

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
teacher_forcing = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, rnn_type, num_layers, max_length=MAX_LENGTH):

  if rnn_type.__name__ == 'GRU':
      encoder_hidden = encoder.initHidden()
  else:
      encoder_hidden = (encoder.initHidden(), encoder.initHidden())

  encoder_optimizer.zero_grad()
  decoder_optimizer.zero_grad()

  input_length = input_tensor.size(0)
  target_length = target_tensor.size(0)

  encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

  loss = 0

  for ei in range(input_length):
    encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
    encoder_outputs[ei] = encoder_output[0, 0]

  decoder_input = torch.tensor([[SOS_token]], device=device)

  decoder_hidden = encoder_hidden

  use_teacher_forsing = True if random.random() < teacher_forcing else False

  if use_teacher_forsing:
    for di in range(target_length):
      decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
      loss += criterion(decoder_output, target_tensor[di])

      decoder_input = target_tensor[di]

  else:
    for di in range(target_length):
      decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
      loss += criterion(decoder_output, target_tensor[di])

      topv, topi = decoder_output.topk(1)
      decoder_input = topi.squeeze().detach()
      if decoder_input.item() == EOS_token:
        break

  loss.backward()

  encoder_optimizer.step()

  decoder_optimizer.step()

  return loss.item() / target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def trainIters(encoder, decoder, n_iters, rnn_type, num_layers=1, print_every=1000, plot_every=100, learning_rate=0.001):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion, rnn_type, num_layers)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
def evaluate(encoder, decoder, sentence, rnn_type, num_layers, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]


        if rnn_type.__name__ == 'GRU':
            encoder_hidden = encoder.initHidden()
        else:
            encoder_hidden = (encoder.initHidden(), encoder.initHidden())

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [ ]:
def evaluateRandomly(encoder, decoder, rnn_type, num_layers, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], rnn_type, num_layers)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
# 1 Layer GRU
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size, nn.GRU, 1).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words, nn.GRU, 1).to(device)

trainIters(encoder1, decoder1, 100000, nn.GRU, num_layers=1, print_every=5000)

1m 24s (- 26m 38s) (5000 5%) 4.6315
2m 43s (- 24m 27s) (10000 10%) 4.2509
4m 2s (- 22m 53s) (15000 15%) 4.0556
5m 21s (- 21m 27s) (20000 20%) 3.9162
6m 41s (- 20m 5s) (25000 25%) 3.7946
8m 2s (- 18m 44s) (30000 30%) 3.7622
9m 22s (- 17m 25s) (35000 35%) 3.6847
10m 42s (- 16m 4s) (40000 40%) 3.6044
12m 3s (- 14m 44s) (45000 45%) 3.5588
13m 24s (- 13m 24s) (50000 50%) 3.5064
14m 45s (- 12m 4s) (55000 55%) 3.4483
16m 6s (- 10m 44s) (60000 60%) 3.4123
17m 27s (- 9m 23s) (65000 65%) 3.3665
18m 47s (- 8m 3s) (70000 70%) 3.3326
20m 7s (- 6m 42s) (75000 75%) 3.2973
21m 28s (- 5m 22s) (80000 80%) 3.2432
22m 48s (- 4m 1s) (85000 85%) 3.1768
24m 10s (- 2m 41s) (90000 90%) 3.1483
25m 30s (- 1m 20s) (95000 95%) 3.1383
26m 50s (- 0m 0s) (100000 100%) 3.0916


In [ ]:
evaluateRandomly(encoder1, decoder1, nn.GRU, 1)

> you re on the list aren t you ?
= ты же есть в списке ?
< вы ведь на ? ? <EOS>

> i m sure tom will agree to help .
= я уверен что том согласится помочь .
< я уверен что том том . . <EOS>

> i m different from tom .
= я не такои как том .
< я уже что том том . <EOS>

> i m not laughing .
= я не смеюсь .
< я не не . <EOS>

> we re not finished yet .
= мы еще не закончили .
< мы не не не . <EOS>

> he is absorbed in reading detective novels .
= он погружен в чтение детективов .
< он в в в . . <EOS>

> i m pretty sure .
= я вполне уверен .
< я уверен уверен . <EOS>

> i m sure she ll leave early .
= я уверена что она рано уидет .
< я уверен уверен в этом . <EOS>

> i m sure you are wrong .
= я уверен что ты ошибаешься .
< я уверен что вы не . <EOS>

> you re hopeless .
= ты безнадежен !
< вы хорошии . <EOS>



In [ ]:
# 2 Layers GRU
hidden_size = 256
encoder2 = EncoderRNN(input_lang.n_words, hidden_size, nn.GRU, num_layers=2).to(device)
decoder2 = DecoderRNN(hidden_size, output_lang.n_words, nn.GRU, num_layers=2).to(device)

trainIters(encoder2, decoder2, 100000, nn.GRU, num_layers=2, print_every=5000)

1m 30s (- 28m 41s) (5000 5%) 4.8070
2m 55s (- 26m 21s) (10000 10%) 4.2739
4m 22s (- 24m 47s) (15000 15%) 4.1288
5m 49s (- 23m 16s) (20000 20%) 4.0151
7m 16s (- 21m 48s) (25000 25%) 3.9239
8m 43s (- 20m 21s) (30000 30%) 3.8598
10m 10s (- 18m 54s) (35000 35%) 3.8195
11m 38s (- 17m 27s) (40000 40%) 3.7707
13m 5s (- 16m 0s) (45000 45%) 3.7037
14m 32s (- 14m 32s) (50000 50%) 3.6506
16m 0s (- 13m 5s) (55000 55%) 3.6196
17m 29s (- 11m 39s) (60000 60%) 3.5905
18m 58s (- 10m 13s) (65000 65%) 3.5466
20m 25s (- 8m 45s) (70000 70%) 3.4933
21m 52s (- 7m 17s) (75000 75%) 3.4683
23m 20s (- 5m 50s) (80000 80%) 3.4126
24m 47s (- 4m 22s) (85000 85%) 3.3938
26m 15s (- 2m 55s) (90000 90%) 3.3673
27m 42s (- 1m 27s) (95000 95%) 3.3220
29m 9s (- 0m 0s) (100000 100%) 3.2925


In [ ]:
evaluateRandomly(encoder2, decoder2, nn.GRU, 2)

> she s sure to succeed .
= к неи непременно придет успех .
< она очень это . . <EOS>

> i m starting to lose hope .
= я начинаю терять надежду .
< я собираюсь в . . <EOS>

> i m not good at doing that .
= у меня это не очень хорошо получается .
< я не не в в . . <EOS>

> i m a bit shy .
= я немного застенчив .
< я в . . <EOS>

> you re completely healthy .
= ты совершенно здоров .
< вы слишком . . <EOS>

> she s worried about your safety .
= она волнуется за твою безопасность .
< она на на на . . <EOS>

> i m trying to concentrate .
= я пытаюсь сосредоточиться .
< я собираюсь . . <EOS>

> you re a little weird .
= ты немного странныи .
< ты хорошии . . <EOS>

> i m happy to see you here .
= я рад видеть вас здесь .
< я так рад что ты . . <EOS>

> you re not my friend .
= ты мне не друг .
< вы не не . . <EOS>



In [ ]:
# 1 Layer GRU scalar
hidden_size = 256
encoder3 = EncoderRNN(input_lang.n_words, hidden_size, nn.GRU, num_layers=1).to(device)
decoder3 = DecoderRNN_scalar(hidden_size, output_lang.n_words, nn.GRU, num_layers=1).to(device)

trainIters(encoder3, decoder3, 100000, nn.GRU, num_layers=2, print_every=5000)

1m 22s (- 26m 2s) (5000 5%) 4.6154
2m 40s (- 24m 0s) (10000 10%) 4.2843
3m 58s (- 22m 30s) (15000 15%) 4.1346
5m 16s (- 21m 6s) (20000 20%) 4.0022
6m 34s (- 19m 43s) (25000 25%) 3.8671
7m 53s (- 18m 24s) (30000 30%) 3.7766
9m 12s (- 17m 5s) (35000 35%) 3.7245
10m 32s (- 15m 48s) (40000 40%) 3.6547
11m 52s (- 14m 31s) (45000 45%) 3.5895
13m 11s (- 13m 11s) (50000 50%) 3.5443
14m 30s (- 11m 52s) (55000 55%) 3.4943
15m 50s (- 10m 33s) (60000 60%) 3.4526
17m 9s (- 9m 14s) (65000 65%) 3.3983
18m 28s (- 7m 55s) (70000 70%) 3.3778
19m 47s (- 6m 35s) (75000 75%) 3.3181
21m 6s (- 5m 16s) (80000 80%) 3.2830
22m 25s (- 3m 57s) (85000 85%) 3.2756
23m 43s (- 2m 38s) (90000 90%) 3.1765
25m 3s (- 1m 19s) (95000 95%) 3.1902
26m 22s (- 0m 0s) (100000 100%) 3.1123


In [ ]:
evaluateRandomly(encoder3, decoder3, nn.GRU, 1)

> i m not sure that i m right .
= я не уверен что прав .
< я не уверен что я . <EOS>

> you re going to ruin your eyes .
= вы себе глаза испортите .
< ты в в . . <EOS>

> i m glad tom was caught .
= я рад что тома поимали .
< я рад что том это . <EOS>

> i m very grateful .
= я очень благодарна .
< я очень хорошо . <EOS>

> we re able to do that .
= мы в состоянии с этим справиться .
< мы будем это . . <EOS>

> you re on the wrong road .
= вы неправильно едете .
< ты не в . . <EOS>

> we re coming back tonight .
= мы возвращаемся сегодня вечером .
< мы оба в . . <EOS>

> i m not a good cook .
= я не очень хорошо готовлю .
< я не очень хорошо . <EOS>

> i m starting a new life .
= я начинаю новую жизнь .
< я немного на . . <EOS>

> i m tired of translating .
= мне надоело переводить .
< я устал устал . <EOS>



In [ ]:
# 2 Layers GRU scalar
hidden_size = 256
encoder4 = EncoderRNN(input_lang.n_words, hidden_size, nn.GRU, num_layers=2).to(device)
decoder4 = DecoderRNN_scalar(hidden_size, output_lang.n_words, nn.GRU, num_layers=2).to(device)

trainIters(encoder4, decoder4, 100000, nn.GRU, num_layers=2, print_every=5000)

1m 26s (- 27m 28s) (5000 5%) 4.7221
2m 49s (- 25m 22s) (10000 10%) 4.2679
4m 14s (- 24m 0s) (15000 15%) 4.1761
5m 38s (- 22m 35s) (20000 20%) 4.0622
7m 4s (- 21m 13s) (25000 25%) 3.9572
8m 30s (- 19m 50s) (30000 30%) 3.8751
9m 55s (- 18m 26s) (35000 35%) 3.8170
11m 21s (- 17m 2s) (40000 40%) 3.7440
12m 49s (- 15m 39s) (45000 45%) 3.7028
14m 16s (- 14m 16s) (50000 50%) 3.6457
15m 43s (- 12m 51s) (55000 55%) 3.6290
17m 8s (- 11m 25s) (60000 60%) 3.5543
18m 34s (- 10m 0s) (65000 65%) 3.5459
20m 1s (- 8m 34s) (70000 70%) 3.4963
21m 27s (- 7m 9s) (75000 75%) 3.4742
22m 53s (- 5m 43s) (80000 80%) 3.3994
24m 20s (- 4m 17s) (85000 85%) 3.3766
25m 47s (- 2m 51s) (90000 90%) 3.3586
27m 13s (- 1m 25s) (95000 95%) 3.3047
28m 39s (- 0m 0s) (100000 100%) 3.2769


In [ ]:
evaluateRandomly(encoder4, decoder4, nn.GRU, 2)

> i m not that wild .
= я не настолько буиныи .
< я не не . . <EOS>

> i m sure a solution can be found .
= я уверен что решение может быть наидено .
< я уверен что том не . . <EOS>

> i m not ready to die .
= я не готов умирать .
< я не собираюсь не . <EOS>

> we re almost ready .
= мы почти готовы .
< мы все . . <EOS>

> i m quite young .
= я довольно молод .
< я не устал . <EOS>

> i am not coming today .
= я сегодня не приду .
< я не не . . <EOS>

> i m looking for a sweater .
= я ищу свитер .
< я ищу . <EOS>

> i m tired of losing .
= я устал проигрывать .
< я устал в . <EOS>

> i m sorry i forgot your name .
= простите я забыл ваше имя .
< я что я я . . . <EOS>

> i m glad you didn t do it .
= я рад что вы этого не сделали .
< я рад что ты не не . . <EOS>

